In [1]:
from numpy import random
file="predict1226/protein_test.txt"
def load_data(filename):
    returnval = []
    with open(filename) as file1:
        content2 = file1.read().replace("\n", " ").split(">")
        content = [a.split(" ", 1) for a in content2]
        for a in content[1:]:
            if a != "" and a!= [""] and a != []:
                returnval.append([a[0], a[1].replace(" ", "")])
    return returnval
dataset = load_data(file)
amino_acid="ARNDCQEGHILKMFPSTWYV"
def get_amino_acid_frequencies(seq):
    return [ seq.count(base)/len(seq)
        for base in amino_acid]
import csv
for data in dataset:
    frequencies = get_amino_acid_frequencies(data[1])
    with open("predict1226/1_feature_amino_acid_freq.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow(frequencies)

In [2]:
import scipy.io as sio
import numpy as np
import pandas as pd
import csv
pssm_file="predict1226/protein_test.mat"
load_data = sio.loadmat(pssm_file)
n=3
lamd=1
for pssm in load_data['train_pssm'][0]:
    L = len(pssm)
    length=int(L/n)
    if(L%n==0):
        L1=L2=length
    else:
        L1=length
        L2=L-L1*(n-1)
    feature1=[]
    for k in range(1,n):
        temp_fea=[]
        for j in range(0,20):
            temp=0
            for i in range((k-1)*L1,k*L1):
                temp+=pssm[i][j]
            temp_=temp/L1
            temp_fea.append(temp_)
        feature1.extend(temp_fea)
    feature2=[]
    for k in range(1,n):
        for lam in range(1,lamd+1):
            temp_fea=[]
            for j in range(0,20):
                temp=0
                for i in range((k-1)*L1,k*L1-lam):
                    temp+=(pssm[i][j]-pssm[i+lam][j])**2
                temp_=temp/(L1-lam)
                temp_fea.append(temp_)
            feature2.extend(temp_fea)
    feature1_n=[]
    for j in range(0,20):
        temp=0
        for i in range((n-1)*L1,(n-1)*L1+L2):
            temp+=pssm[i][j]
        temp_=temp/L2
        feature1_n.append(temp_)
    feature2_n=[]
    for lam in range(1,lamd+1):
        for j in range(0,20):
            temp=0
            for i in range((n-1)*L1,(n-1)*L1+L2-lam):
                temp+=(pssm[i][j]-pssm[i+lam][j])**2
            temp_=temp/(L2-lam)
            feature2_n.append(temp_)   
    feature=[]
    feature.extend(feature1) # 40 
    feature.extend(feature2) # 40
    feature.extend(feature1_n) # 20
    feature.extend(feature2_n) # 20
    #print('feature......',feature) 
    #print(len(feature)) # 120
    with open("predict1226/2_feature_LocalDPP.csv","a+",newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(feature)

In [3]:
import scipy.io as sio
import numpy as np
pssm_file = 'predict1226/protein_test.mat'
load_data = sio.loadmat(pssm_file)
for pssm in load_data['train_pssm'][0]:
    L = len(pssm)
    LG = 5
    SDT=[]
    for lg in range(1,LG+1):
        SDT_lg=[]
        for i in range(0,20):
            temp=0
            for j in range(0,L-lg):
                temp+=pssm[j][i]*pssm[j+lg][i]
            temp=temp/(L-lg)
            SDT_lg.append(temp)
        SDT.extend(SDT_lg)
    with open("predict1226/3_feature_PSSMDT_SDT.csv","a+",newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(SDT)
for pssm in load_data['train_pssm'][0]:
    L = len(pssm)
    LG = 5
    DDT=[]
    for lg in range(1,LG+1):
        DDT_lg=[]
        temp=0
        for i1 in range(0,20):
            for i2 in range(0,20):
                if i1==i2:
                    continue
                for j in range(0,L-lg):
                    temp+=pssm[j][i1]*pssm[j+lg][i2]
                temp=temp/(L-lg)
                DDT_lg.append(temp)
        DDT.extend(DDT_lg)
    with open("predict1226/3_feature_PSSMDT_DDT.csv","a+",newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(DDT)

In [4]:
file="predict1226/protein_test.txt"
amino_acid_10=["KR","LV","RK","GA","VL","AG","AV","EK","GD","AL"]
def load_data(filename):
    returnval = []
    with open(filename) as file1:
        content2 = file1.read().replace("\n", " ").split(">")
        content = [a.split(" ", 1) for a in content2]
        for a in content[1:]:
            if a != "" and a!= [""] and a != []:
                returnval.append([a[0], a[1].replace(" ", "")])
    return returnval
dataset = load_data(file)
import re
def get_amino_acid_frequencies_top_10(seq):
    res = []
    for base in amino_acid_10:
        reg = re.compile("(?="+base+")")
        length = len(reg.findall(seq))
        res.append(length/(len(seq)))
    return res
import csv
for data in dataset:
    frequencies = get_amino_acid_frequencies_top_10(data[1])
    with open("predict1226/4_feature_amino_acid_freq_top_10_2.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow(frequencies)

In [2]:
import codecs
from itertools import *
import numpy as np
def load_predict_data(filename):
    file = codecs.open(filename,'r','utf-8')
    data = []
    for line in islice(file,0,None):
        line = line.strip().split(',')
        #print ("reading data....")
        data.append([float(i) for i in line[1:]])
    x = np.array(data)
    return x

In [10]:
from sklearn.externals import joblib
predict_x = load_predict_data("predict1226/feature_predict_all.csv")
gdbt_model = joblib.load("predict1226/gbdt.model")
y_pred = gdbt_model.predict(predict_x)
y_preprob = gdbt_model.predict_proba(predict_x)[:,1]
print (y_pred)  
print (y_preprob)

['1' '1' '0' '1' '0' '1' '1' '1' '1' '0' '1' '1' '1' '1' '0' '0' '1' '1'
 '1' '0' '0' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1'
 '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '0' '0' '1' '1' '1' '0' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '0' '1' '1' '1' '0'
 '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1'
 '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '0'
 '0' '0' '0' '0' '0' '1' '1' '1' '0' '1' '1' '1' '1' '0' '1' '0' '1' '0'
 '1' '1' '0' '0' '1' '1' '0' '0' '1' '0' '0' '1' '0' '0' '1' '0' '0' '1'
 '0' '0' '0' '0' '1' '1' '0' '1' '1' '1' '0' '0' '0' '1' '1' '0' '1' '0'
 '1' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0' '1' '1' '1' '1' '1'
 '1' '1' '1' '0' '0' '1']
[ 0.86842993  0.780053    0.49336591  0.94972933  0.4873381   0.74861346
  0.93685305  0.64163135  0.89028631  0.43460625  0.50021653  0.9423178
  0.91520246  0.56671787  0.3629235   0.04158448  0.94857152  0.75553737
  0.56241172  0.44918839  

In [11]:
for y in y_pred:
    with open("predict1226/result.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow(y)

In [4]:
from sklearn.externals import joblib
predict_x = load_predict_data("test1226/feature_all.csv")
rf_model = joblib.load("predict1226_2/rf.model")
y_pred = rf_model.predict(predict_x)
y_preprob = rf_model.predict_proba(predict_x)[:,1]
print (y_pred)  
print (y_preprob)

['1' '1' '1' '1' '0' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1'
 '1' '0' '0' '1' '1' '1' '1' '0' '1' '0' '1' '0' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '0' '1' '1' '1' '1' '1' '0'
 '1' '1' '1' '0' '1' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1'
 '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0'
 '0' '0' '0' '0' '0' '1' '0' '1' '0' '1' '0' '0' '1' '0' '1' '0' '1' '0'
 '1' '1' '0' '1' '1' '1' '0' '1' '1' '0' '0' '1' '0' '0' '0' '0' '0' '1'
 '0' '0' '0' '0' '1' '1' '0' '1' '1' '1' '0' '0' '1' '1' '1' '0' '1' '1'
 '1' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '1' '0' '1' '1' '0' '1' '1'
 '1' '1' '1' '1' '1' '1']
[ 0.81  0.83  0.85  0.88  0.37  0.51  0.44  0.95  0.76  0.55  0.51  0.96
  0.9   0.64  0.6   0.25  0.99  0.88  0.62  0.38  0.44  0.93  0.99  0.91
  0.68  0.37  0.57  0.22  0.7   0.08  0.86  0.93  0.96  0.91  0.93  0.94
  0.97  0.97  0.94  0.94 

In [8]:
import csv
for y in y_pred:
    with open("test1226/result_1226.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow(y)

写文件，使得能够一一对应

In [12]:
import codecs
def load(filename):
    file = codecs.open(filename,'r','utf-8')
    data = []
    for line in islice(file,0,None):
        line = line.strip().split(',')
        #print ("reading data....")
        print(line[0],line[1])

In [13]:
load("test1226/result_1226.csv")

2L8DA 1
2LI6A 1
2LJ6A 1
2LUYA 1
2LV4A 0
2LV9A 1
2LVFA 0
2LVSA 1
2LW1A 1
2LXUA 1
2LY4B 1
2LYHA 1
2LYJA 1
2LYVA 1
2LZEA 1
2M0YA 0
2MA1A 1
2YMBA 1
2YMVA 1
2YNQA 0
2YO3A 0
3AXJB 1
3B0CT 1
3J20A 1
3J21P 1
3J21R 0
3J21a 1
3J21k 0
3J21l 1
3J27A 0
3Q8DA 1
3QODA 1
3QU3A 1
3R0JA 1
3R4KA 1
3RCOA 1
3RH2A 1
3RLOA 1
3S4WA 1
3S4WB 1
3S5RA 1
3SIAA 1
3TB6A 1
3TEKA 1
3TOCA 1
3TRBA 1
3TUOA 1
3U4VA 1
3U4ZA 1
3U50C 1
3ULJA 1
3ULXA 1
3UQZA 0
3UWXB 1
3UX8A 1
3V4GA 1
3V68A 1
3V9RA 1
3V9RB 1
3VH5A 1
3VH5W 1
3VIBA 1
3VK0A 1
3VPRA 1
3VYSC 0
3VYWA 0
3VZHA 1
3VZIA 1
3W03A 1
3W03C 1
3W1SC 1
3ZDWA 0
3ZQJA 1
4A11A 1
4A11B 1
4ABXA 0
4ACOA 1
4AD8A 1
4ATHA 1
4AXNA 0
4AXQA 0
4AY9B 0
4AYFB 0
4AZ1A 0
4B0EA 0
4B29A 0
4B2XA 0
4B4TI 1
4B4TO 1
4B6AM 1
4B9JA 0
4B9KC 1
4BBKA 1
4BHXA 1
4BJ1A 1
4BLFA 1
4D8JA 1
4DAMA 1
4DG7A 1
4DKYA 1
4DRAE 1
4E1RA 1
4EOGA 1
4EQ6A 1
4EQ6B 1
4EXWA 1
4FE7A 1
4FIPA 0
4FJ6A 0
4FL4C 0
4FLTA 0
4FM4A 0
4FM4B 0
4FMWA 1
4FMZA 0
4FNHA 1
4FQDA 0
4FW2A 1
4FY11 0
4FZQA 0
4G12A 1
4G1EB 0
4G4KA 1
4G4SP 0
4G56B 1


In [19]:
filename1="test1226/result_1226.csv"
file1 = codecs.open(filename1,'r','utf-8')
filename2="test1226/predict_order.csv"
file2 = codecs.open(filename2,"r","utf-8")
data1 = []
data2 = []
for line2 in islice(file2,0,None):
    line2 = line2.strip().split(',')
    data2.append(line2[0])
for line1 in islice(file1,0,None):
    line1 = line1.strip().split(',')
    data1.append([line1[0],line1[1]])

In [20]:
data1

[['2L8DA', '1'],
 ['2LI6A', '1'],
 ['2LJ6A', '1'],
 ['2LUYA', '1'],
 ['2LV4A', '0'],
 ['2LV9A', '1'],
 ['2LVFA', '0'],
 ['2LVSA', '1'],
 ['2LW1A', '1'],
 ['2LXUA', '1'],
 ['2LY4B', '1'],
 ['2LYHA', '1'],
 ['2LYJA', '1'],
 ['2LYVA', '1'],
 ['2LZEA', '1'],
 ['2M0YA', '0'],
 ['2MA1A', '1'],
 ['2YMBA', '1'],
 ['2YMVA', '1'],
 ['2YNQA', '0'],
 ['2YO3A', '0'],
 ['3AXJB', '1'],
 ['3B0CT', '1'],
 ['3J20A', '1'],
 ['3J21P', '1'],
 ['3J21R', '0'],
 ['3J21a', '1'],
 ['3J21k', '0'],
 ['3J21l', '1'],
 ['3J27A', '0'],
 ['3Q8DA', '1'],
 ['3QODA', '1'],
 ['3QU3A', '1'],
 ['3R0JA', '1'],
 ['3R4KA', '1'],
 ['3RCOA', '1'],
 ['3RH2A', '1'],
 ['3RLOA', '1'],
 ['3S4WA', '1'],
 ['3S4WB', '1'],
 ['3S5RA', '1'],
 ['3SIAA', '1'],
 ['3TB6A', '1'],
 ['3TEKA', '1'],
 ['3TOCA', '1'],
 ['3TRBA', '1'],
 ['3TUOA', '1'],
 ['3U4VA', '1'],
 ['3U4ZA', '1'],
 ['3U50C', '1'],
 ['3ULJA', '1'],
 ['3ULXA', '1'],
 ['3UQZA', '0'],
 ['3UWXB', '1'],
 ['3UX8A', '1'],
 ['3V4GA', '1'],
 ['3V68A', '1'],
 ['3V9RA', '1'],
 ['3V9RB', '1'

In [21]:
data2

['2MA1A',
 '4LC2A',
 '4BLFA',
 '4BJ1A',
 '4BHXA',
 '4JQFA',
 '4JOIC',
 '4JOIA',
 '4JOLA',
 '4JLXA',
 '4IDUA',
 '4ICGC',
 '4I99C',
 '4I99A',
 '4I1KA',
 '4HPZA',
 '3W03C',
 '3W03A',
 '4HLXA',
 '3VZIA',
 '3VZHA',
 '4HIDA',
 '4HD0A',
 '4H7AA',
 '4H79A',
 '2LYJA',
 '2LYHA',
 '4GS3A',
 '2LW1A',
 '4G6DB',
 '4G6DA',
 '4G4KA',
 '2LVSA',
 '4G12A',
 '4FW2A',
 '2LUYA',
 '4FE7A',
 '4ATHA',
 '4EXWA',
 '4EQ6A',
 '4EQ6B',
 '4EOGA',
 '3VPRA',
 '4E1RA',
 '4DRAE',
 '4DKYA',
 '4DG7A',
 '4DAMA',
 '4D8JA',
 '3V9RB',
 '3V9RA',
 '4AD8A',
 '3V68A',
 '4ACOA',
 '3V4GA',
 '4ABXA',
 '3UX8A',
 '3UWXB',
 '3UQZA',
 '3ULXA',
 '3ULJA',
 '3VK0A',
 '3U50C',
 '3U4VA',
 '3U4ZA',
 '3VIBA',
 '3TUOA',
 '4A11A',
 '4A11B',
 '3TRBA',
 '2LJ6A',
 '3TOCA',
 '2LI6A',
 '3VH5A',
 '3VH5W',
 '3TEKA',
 '3TB6A',
 '3SIAA',
 '3ZQJA',
 '3B0CT',
 '3S5RA',
 '3S4WB',
 '3S4WA',
 '3RLOA',
 '3RH2A',
 '3AXJB',
 '3RCOA',
 '3R4KA',
 '3R0JA',
 '3QU3A',
 '3QODA',
 '2L8DA',
 '3Q8DA',
 '4IAOC',
 '4I8DA',
 '3ZDWA',
 '3W1SC',
 '4I1DA',
 '4HY3A',
 '4HWXA',


In [27]:
result=[]
for a in data2:
    #print (a)
    for b in data1:
        if(a==b[0]):
            print (b[1])
            result.append(b[1])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
1
1
0
0
0
1
1
1
0
1
0
0
1
0
1
1
1
1
1
0
0
0
0
0
1
0
0
0
1
1
0
1
1
0
0
0
1
1
0
1
1
0
1
0
1
1
1
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0


In [28]:
len(result)

186

In [29]:
for y_hat in result:
    with open("test1226/predict_result.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow(y_hat)

In [30]:
result

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0'

In [35]:
txtName = "test1226/result.txt"
for y in result:
    with open(txtName,"a+") as f:
        content = y+'\n'
        f.write(content)

In [40]:
data=[]
with open("test1226/result.csv") as f:
    reader = f.read()
    line = reader.split(",")
    print (line)

['2MA1A', '1\n4LC2A', '1\n4BLFA', '1\n4BJ1A', '1\n4BHXA', '1\n4JQFA', '1\n4JOIC', '1\n4JOIA', '1\n4JOLA', '1\n4JLXA', '1\n4IDUA', '1\n4ICGC', '1\n4I99C', '1\n4I99A', '1\n4I1KA', '1\n4HPZA', '1\n3W03C', '1\n3W03A', '1\n4HLXA', '1\n3VZIA', '1\n3VZHA', '1\n4HIDA', '1\n4HD0A', '1\n4H7AA', '1\n4H79A', '1\n2LYJA', '1\n2LYHA', '1\n4GS3A', '1\n2LW1A', '1\n4G6DB', '1\n4G6DA', '1\n4G4KA', '1\n2LVSA', '1\n4G12A', '1\n4FW2A', '1\n2LUYA', '1\n4FE7A', '1\n4ATHA', '1\n4EXWA', '1\n4EQ6A', '1\n4EQ6B', '1\n4EOGA', '1\n3VPRA', '1\n4E1RA', '1\n4DRAE', '1\n4DKYA', '1\n4DG7A', '1\n4DAMA', '1\n4D8JA', '1\n3V9RB', '1\n3V9RA', '1\n4AD8A', '1\n3V68A', '1\n4ACOA', '1\n3V4GA', '1\n4ABXA', '0\n3UX8A', '1\n3UWXB', '1\n3UQZA', '0\n3ULXA', '1\n3ULJA', '1\n3VK0A', '1\n3U50C', '1\n3U4VA', '1\n3U4ZA', '1\n3VIBA', '1\n3TUOA', '1\n4A11A', '1\n4A11B', '1\n3TRBA', '1\n2LJ6A', '1\n3TOCA', '1\n2LI6A', '1\n3VH5A', '1\n3VH5W', '1\n3TEKA', '1\n3TB6A', '1\n3SIAA', '1\n3ZQJA', '1\n3B0CT', '1\n3S5RA', '1\n3S4WB', '1\n3S4WA', '1\n3R

In [44]:
filename="test1226/result.csv"
file = codecs.open(filename,'r','utf-8')
data=[]
for line in islice(file,0,None):
    line = line.strip().split(',')
    data.append([line[0],line[1]])
print (data)

[['2MA1A', '1'], ['4LC2A', '1'], ['4BLFA', '1'], ['4BJ1A', '1'], ['4BHXA', '1'], ['4JQFA', '1'], ['4JOIC', '1'], ['4JOIA', '1'], ['4JOLA', '1'], ['4JLXA', '1'], ['4IDUA', '1'], ['4ICGC', '1'], ['4I99C', '1'], ['4I99A', '1'], ['4I1KA', '1'], ['4HPZA', '1'], ['3W03C', '1'], ['3W03A', '1'], ['4HLXA', '1'], ['3VZIA', '1'], ['3VZHA', '1'], ['4HIDA', '1'], ['4HD0A', '1'], ['4H7AA', '1'], ['4H79A', '1'], ['2LYJA', '1'], ['2LYHA', '1'], ['4GS3A', '1'], ['2LW1A', '1'], ['4G6DB', '1'], ['4G6DA', '1'], ['4G4KA', '1'], ['2LVSA', '1'], ['4G12A', '1'], ['4FW2A', '1'], ['2LUYA', '1'], ['4FE7A', '1'], ['4ATHA', '1'], ['4EXWA', '1'], ['4EQ6A', '1'], ['4EQ6B', '1'], ['4EOGA', '1'], ['3VPRA', '1'], ['4E1RA', '1'], ['4DRAE', '1'], ['4DKYA', '1'], ['4DG7A', '1'], ['4DAMA', '1'], ['4D8JA', '1'], ['3V9RB', '1'], ['3V9RA', '1'], ['4AD8A', '1'], ['3V68A', '1'], ['4ACOA', '1'], ['3V4GA', '1'], ['4ABXA', '0'], ['3UX8A', '1'], ['3UWXB', '1'], ['3UQZA', '0'], ['3ULXA', '1'], ['3ULJA', '1'], ['3VK0A', '1'], ['3U50C

In [47]:
for a in data:
    print (a)

['2MA1A', '1']
['4LC2A', '1']
['4BLFA', '1']
['4BJ1A', '1']
['4BHXA', '1']
['4JQFA', '1']
['4JOIC', '1']
['4JOIA', '1']
['4JOLA', '1']
['4JLXA', '1']
['4IDUA', '1']
['4ICGC', '1']
['4I99C', '1']
['4I99A', '1']
['4I1KA', '1']
['4HPZA', '1']
['3W03C', '1']
['3W03A', '1']
['4HLXA', '1']
['3VZIA', '1']
['3VZHA', '1']
['4HIDA', '1']
['4HD0A', '1']
['4H7AA', '1']
['4H79A', '1']
['2LYJA', '1']
['2LYHA', '1']
['4GS3A', '1']
['2LW1A', '1']
['4G6DB', '1']
['4G6DA', '1']
['4G4KA', '1']
['2LVSA', '1']
['4G12A', '1']
['4FW2A', '1']
['2LUYA', '1']
['4FE7A', '1']
['4ATHA', '1']
['4EXWA', '1']
['4EQ6A', '1']
['4EQ6B', '1']
['4EOGA', '1']
['3VPRA', '1']
['4E1RA', '1']
['4DRAE', '1']
['4DKYA', '1']
['4DG7A', '1']
['4DAMA', '1']
['4D8JA', '1']
['3V9RB', '1']
['3V9RA', '1']
['4AD8A', '1']
['3V68A', '1']
['4ACOA', '1']
['3V4GA', '1']
['4ABXA', '0']
['3UX8A', '1']
['3UWXB', '1']
['3UQZA', '0']
['3ULXA', '1']
['3ULJA', '1']
['3VK0A', '1']
['3U50C', '1']
['3U4VA', '1']
['3U4ZA', '1']
['3VIBA', '1']
['3TUOA', 

In [50]:
def load_data(negative):
    returnval = []
    with open(negative) as file1:
        content2 = file1.read().replace("\n", " ").split(">")
        content = [a.split(" ", 1) for a in content2]
        for a in content[1:]:
            if a != "" and a!= [""] and a != []:
                returnval.append(a[0])
    return returnval

In [53]:
order = load_data("test1226/protein_test.txt")

In [54]:
for a in data:
    print (a[0],a[1])
for b in order:
    print(b)

2MA1A 1
4LC2A 1
4BLFA 1
4BJ1A 1
4BHXA 1
4JQFA 1
4JOIC 1
4JOIA 1
4JOLA 1
4JLXA 1
4IDUA 1
4ICGC 1
4I99C 1
4I99A 1
4I1KA 1
4HPZA 1
3W03C 1
3W03A 1
4HLXA 1
3VZIA 1
3VZHA 1
4HIDA 1
4HD0A 1
4H7AA 1
4H79A 1
2LYJA 1
2LYHA 1
4GS3A 1
2LW1A 1
4G6DB 1
4G6DA 1
4G4KA 1
2LVSA 1
4G12A 1
4FW2A 1
2LUYA 1
4FE7A 1
4ATHA 1
4EXWA 1
4EQ6A 1
4EQ6B 1
4EOGA 1
3VPRA 1
4E1RA 1
4DRAE 1
4DKYA 1
4DG7A 1
4DAMA 1
4D8JA 1
3V9RB 1
3V9RA 1
4AD8A 1
3V68A 1
4ACOA 1
3V4GA 1
4ABXA 0
3UX8A 1
3UWXB 1
3UQZA 0
3ULXA 1
3ULJA 1
3VK0A 1
3U50C 1
3U4VA 1
3U4ZA 1
3VIBA 1
3TUOA 1
4A11A 1
4A11B 1
3TRBA 1
2LJ6A 1
3TOCA 1
2LI6A 1
3VH5A 1
3VH5W 1
3TEKA 1
3TB6A 1
3SIAA 1
3ZQJA 1
3B0CT 1
3S5RA 1
3S4WB 1
3S4WA 1
3RLOA 1
3RH2A 1
3AXJB 1
3RCOA 1
3R4KA 1
3R0JA 1
3QU3A 1
3QODA 1
2L8DA 1
3Q8DA 1
4IAOC 0
4I8DA 0
3ZDWA 0
3W1SC 1
4I1DA 0
4HY3A 0
4HWXA 0
2M0YA 0
4HVTA 0
4HWGA 0
4HQFA 0
4HPQC 1
2YO3A 0
4HN8A 0
4HN9A 0
2YNQA 0
2YMVA 1
2YMBA 1
4HGAB 1
4HESA 0
3VYWA 0
3VYSC 0
2LZEA 1
4HD1A 1
4HCEA 1
4HC6A 0
4HAMA 1
4HAOA 0
3J27A 0
4BBKA 1
4H63F 0
2LYVA 1


In [55]:
res = []
for b in order:
    for a in data:
        if(b==a[0]):
            res.append(a[1])
print(res)

['1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1']


In [56]:
len(res)

186

In [58]:
txtName = "test1226/result_2.txt"
for y in res:
    with open(txtName,"a+") as f:
        content = y+'\n'
        f.write(content)

In [59]:
for b in order:
    print (b)

2LXUA
4FMZA
4IDUA
4G4KA
4FE7A
4GAKA
2LV9A
4HPZA
2L8DA
4AY9B
2LUYA
3R4KA
3W03C
4FZQA
2YO3A
4JLXA
4EQ6A
3TUOA
4JOIA
4H3HB
4A11A
3Q8DA
2LY4B
4H79A
4ACOA
4JQFA
2LW1A
3J21a
4HGAB
3UX8A
4GS3A
2LVFA
3QODA
2LJ6A
4G59A
2LI6A
4G1EB
4FNHA
3U4ZA
3VYSC
4EQ6B
3U50C
4G6DA
4ICGC
4HIDA
4HVTA
4HAOA
3VZIA
3J21k
3V9RB
4GWGA
4HCEA
3SIAA
4B0EA
3TEKA
4GF2A
4B2XA
4D8JA
2LYVA
3J21P
3VZHA
2YMVA
4GNVA
3J21R
4IAOC
4HWGA
3AXJB
4HN9A
4HN8A
4G9JA
3B0CT
4AZ1A
4G6XA
4FM4B
4HD1A
4DG7A
4JOLA
3UWXB
4GICA
2M0YA
4GVWA
4G12A
3RH2A
3U4VA
4G7LA
4GWPF
4EXWA
3TRBA
4ABXA
3R0JA
2LYHA
3V4GA
4GWPG
4G7HC
3W1SC
3S4WA
3W03A
2LYJA
4ATHA
3VH5W
2LV4A
4G6DB
4FMWA
3TB6A
4GIQA
3ZQJA
2MA1A
3VK0A
4H5XA
4FL4C
4HPQC
4I99C
4G9QA
3J27A
3VIBA
4HQFA
4HLXA
2LZEA
4LC2A
4DAMA
2LVSA
3V68A
4I8DA
4I99A
4GGDA
4G4SP
4I1KA
4AD8A
4HAMA
4GOSA
3S5RA
3TOCA
3RCOA
4G56B
3ULXA
3S4WB
4FLTA
4BHXA
4HC6A
3RLOA
4BLFA
4DKYA
4I1DA
4FY11
4JOIC
4E1RA
4HESA
3VPRA
4B6AM
4HY3A
4A11B
4EOGA
3VYWA
4FJ6A
4FM4A
4AYFB
4HWXA
3UQZA
3J21l
3J20A
4B4TO
3ZDWA
4FW2A
4AXQA
3QU3A
4HD0A
4BBK

In [62]:
for b in order:
    with open("test1226/predict_protein_order.csv","a+",newline='') as file:
        writer = csv.writer(file)
        writer.writerow([b])